<a href="https://colab.research.google.com/github/fadlinisasiGit/SkinCan-model/blob/main/skincan_models_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Product-Based-Capstone-Project**

**SkinCan Skin Cancer Detection App**

Final Selected Themes : Human Healthcare & Animal Welfare

Team Member :
*   (ML) M7134F1606 - Muhammad Fadli Ramadhan - Politeknik Negeri Sriwijaya
*   (ML) M2008F0851 - Adhitya Ghiffari Pramudito - Universitas Gadjah Mada
*   (MD) A2191F1821 - Ahmad Ansori Palembani - Universitas Bina Darma
*   (MD) A7191F1820 - Muhammad Fharid Akbar - Universitas Bina Darma
*   (CC) C2322F2819 - Muhammad Mustafa Kamal - Universitas Syiah Kuala
*   (CC) C7457F3068 - Wulan Ayu Rania Sari -Universitas Nahdlatul Ulama Lampung













**Executive Summary :**

At this time there are so many dangers that can lurk human health, even in this era of pandemic, maintaining health is very important for everyone. People should have awareness to maintain health, especially the health of the skin. Therefore, by utilizing machine learning, cloud computing to create a skin cancer detection application by identifying the input image to remind the public the importance of maintaining healthy skin. In this project, We have a problem formulation as follows:

*   How to detect various skin cancers in humans through the application?
*   How to treat skin cancer early?
*   How accurately does this application detect skin cancer?

Our team wants to solve this problem because we want someone who has the potential for cancer to avoid a more serious problem, so that users who use this application can take preventive measures to prevent this problem.

**How did your team come up with this project?**

We want to build an application that can help the public and health workers to detect skin diseases in humans, including skin cancer. This application uses Deep Learning algorithms to predict various skin diseases to achieve maximum accuracy in predicting skin diseases at an early stage. Feature extraction plays a key role in the classification of skin diseases. In addition, Explainable AI is used to interpret decisions made by our machine learning models. With this application, hopefully, it can help users to detect skin diseases early so they can take preventive measures.

**Based on your knowledge and explorations, tell us the Machine Learning Part of your capstone?**

In this project, we’re using Python as main programming language, Tensorflow as mandatory library, Sckit-learn and keras as libraries, AI Platform as platform, TFLite as Tools for Model Deployment to Android, Kaggle as dataset resource, Google Colab as IDE, and create Machine Learning Model as main the feature in our app.

## **Data Collecting**

In [ ]:
# Import Libraries

import tensorflow as tf
import numpy as np
import pandas as pd
import io
import os

from PIL import Image
from glob import glob
import itertools

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, BatchNormalization, Dropout, Dense, MaxPool2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from IPython.display import display
#To see the value of multiple statements at once.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import warnings # tf needs to learn to stfu
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [ ]:
# Checking tensorflow version
print(tf.__version__)

In [ ]:
# install opendataset packages
!pip install opendatasets

# download plotly package
!pip install plotly

In [ ]:
# install paket kaggle
!pip install -q kaggle

# Upload file kaggle.json dari Create New API Token Akun Kaggle
from google.colab import files
files.upload()

In [ ]:
# Download Dataset from Kaggle
import opendatasets as od
dataset_url= 'https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000'
od.download('https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000')

## Create function def

**function for train test splits**

In [ ]:
# define function for train test splits
def train_test_splits(X, Y):
    # Splitting into train and test set
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)
    
    # Prepare data for training and testing the model
    train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range = 10,
                                  width_shift_range = 0.2,
                                  height_shift_range = 0.2,
                                  shear_range = 0.2,
                                  horizontal_flip = True,
                                  vertical_flip = True,
                                  fill_mode = 'nearest')
    train_datagen.fit(X_train)
    test_datagen = ImageDataGenerator(rescale = 1./255)
    test_datagen.fit(X_test)
    return X_train, X_test, Y_train, Y_test

**function for Modeling**

In [ ]:
# define function for create model
def model():
    model = Sequential()
    model.add(Conv2D(16, kernel_size = (3,3), input_shape = (28, 28, 3), activation = 'relu', padding = 'same'))
    model.add(MaxPool2D(pool_size = (2,2)))

    model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same'))
    model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))

    model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu', padding = 'same'))
    model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))
    model.add(Conv2D(128, kernel_size = (3,3), activation = 'relu', padding = 'same'))
    model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))

    model.add(Flatten())
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(7, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

    model.compile(loss = 'sparse_categorical_crossentropy',
                 optimizer = optimizer,
                  metrics = ['accuracy'])
    print(model.summary())
    return model;

**function for training model**

In [ ]:
# Define function for training model
def train_model(model, X_train, Y_train, EPOCHS=25):
    early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, 
                           mode='auto')
                               
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, 
                              verbose=1, mode='auto')
    
    history = model.fit(X_train,
                        Y_train,
                        validation_split=0.2,
                        batch_size = 64,
                        epochs = EPOCHS,
                        callbacks = [reduce_lr, early_stop])
    return history

**function for test model**

In [ ]:
# Define function for model testing
def test_model(model, X_test, Y_test):
    model_acc = model.evaluate(X_test, Y_test, verbose=0)[1]
    print("Test Accuracy: {:.3f}%".format(model_acc * 100))
    y_true = np.array(Y_test)
    y_pred = model.predict(X_test)
    y_pred = np.array(list(map(lambda x: np.argmax(x), y_pred)))
    clr = classification_report(y_true, y_pred, target_names=label_mapping.values())
    print(clr)
    
    sample_data = X_test[:15]
    plt.figure(figsize=(22, 12))
    for i in range(15):
        plt.subplot(3, 5, i + 1)
        plt.imshow(sample_data[i])
        plt.title(label_mapping[y_true[i][0]] + '|' + label_mapping[y_pred[i]])
        plt.axis("off")
    plt.show() 

**function for plotting model accuracy and loss**

In [ ]:
# Define function for training curves plot
def plot_model_training_curve(history):
    fig = make_subplots(rows=1, cols=2, subplot_titles=['Model Accuracy', 'Model Loss'])
    fig.add_trace(
        go.Scatter(
            y=history.history['accuracy'], 
            name='train_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_accuracy'], 
            name='val_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['loss'], 
            name='train_loss'), 
        row=1, col=2)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_loss'], 
            name='val_loss'), 
        row=1, col=2)
    fig.show()

## **Data Understanding**


In [ ]:
# Create directory
base_skin_dir = '/content/skin-cancer-mnist-ham10000'

# Merging images from both folders HAM10000_images_part1.zip and HAM10000_images_part2.zip into one dictionary

imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir,"skin-cancer-mnist-ham10000/", '*', '*.jpg'))}
# create dictionary for type of skin cancer
lesion_type_dict = {
    'nv': 'Melanocytic nevi (nv)',
    'mel': 'Melanoma (mel)',
    'bkl': 'Benign keratosis-like lesions (bkl)',
    'bcc': 'Basal cell carcinoma (bcc)',
    'akiec': 'Actinic keratoses (akiec)',
    'vasc': 'Vascular lesions (vasc)',
    'df': 'Dermatofibroma (df)'
}
# create label mapping
label_mapping = {
    0: 'nv',
    1: 'mel',
    2: 'bkl',
    3: 'bcc',
    4: 'akiec',
    5: 'vasc',
    6: 'df'
}
reverse_label_mapping = dict((value, key) for key, value in label_mapping.items())

In [ ]:
# read the csv data
data = pd.read_csv(os.path.join(base_skin_dir,"skin-cancer-mnist-ham10000/",'HAM10000_metadata.csv'))

In [ ]:
# Now lets see the sample of tile_df to look on newly made columns
data.head()

In [ ]:
# more info about data 
data.describe(exclude=[np.number])

**Data Cleaning**

In [ ]:
# Checking a null data
data.isnull().any().sum()

In [ ]:
# Handling null values
data['age'].fillna(value=int(data['age'].mean()), inplace=True)
# Converting dtype of age to int32
data['age'] = data['age'].astype('int32')

# Adding cell_type and image_path columns
data['cell_type'] = data['dx'].map(lesion_type_dict.get)
data['path'] = data['image_id'].map(imageid_path_dict.get)

In [ ]:
# check a data
data.head()

In [ ]:
# Adding image pixels
data['image_pixel'] = data['path'].map(lambda x: np.asarray(Image.open(x).resize((28,28))))

In [ ]:
# New updates of data
data.head(5)

## **Data Preparation**

In [ ]:
# Displaying 2 images for each label
sample_data = data.groupby('dx').apply(lambda df: df.iloc[:2, [9, 7]])
plt.figure(figsize=(22, 32))
for i in range(14):
    plt.subplot(7, 5, i + 1)
    plt.imshow(np.squeeze(sample_data['image_pixel'][i]))
    img_label = sample_data['cell_type'][i]
    plt.title(img_label)
    plt.axis("off")
plt.show();

In [ ]:
data['label'] = data['dx'].map(reverse_label_mapping.get)

data = data.sort_values('label')
data = data.reset_index()

In [ ]:
counter = 0
frames = [data]
for i in [4,4,11,17,45,52]:
    counter+=1
    index = data[data['label'] == counter].index.values
    df_index = data.iloc[int(min(index)):int(max(index)+1)]
    df_index = df_index.append([df_index]*i, ignore_index = True)
    frames.append(df_index)

In [ ]:
len(frames)

In [ ]:
final_data = pd.concat(frames)

In [ ]:
print(data.shape)
print(final_data.shape)

In [ ]:
# ORIGINAL DATA
# Converting image pixel columnm into required format
X_orig = data['image_pixel'].to_numpy()
X_orig = np.stack(X_orig, axis=0)
Y_orig = np.array(data.iloc[:, -1:])
print(X_orig.shape)
print(Y_orig.shape)

In [ ]:
# AUGMENTED DATA
# Converting image pixel columnm into required format
X_aug = final_data['image_pixel'].to_numpy()
X_aug = np.stack(X_aug, axis=0)
Y_aug = np.array(final_data.iloc[:, -1:])
print(X_aug.shape)
print(Y_aug.shape)

In [ ]:
# For Original Dataset
X_train_orig, X_test_orig, Y_train_orig, Y_test_orig = train_test_splits(X_orig, Y_orig)

In [ ]:
# For Augmentation Dataset
X_train_aug, X_test_aug, Y_train_aug, Y_test_aug = train_test_splits(X_aug, Y_aug)